# nlp-project-vqa


In [1]:
obs_path = "s3://howie-zju/nlp/nlp_project_vqa/nlp-project-vqa/" # howie's path
# obs_path = "s3://nlp-haofeng/nlp_proje|ct_vqa/" # hy's path
# obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # zc's path
# obs_path = "s3://nlp-haofeng/nlp_project_vqa/" # ct's path

In [2]:
import moxing as mox
mox.file.copy_parallel(src_url=obs_path+"mindrecord", dst_url='./mindrecord') 
mox.file.copy_parallel(src_url=obs_path+"preprocess", dst_url='./preprocess')
mox.file.copy_parallel(src_url=obs_path+"utils",      dst_url='./utils')
mox.file.copy_parallel(src_url=obs_path+"model",      dst_url='./model')
mox.file.copy_parallel(src_url=obs_path+"ckpt",       dst_url='./ckpt')
mox.file.copy_parallel(src_url=obs_path+"pretrained/VGG.py", dst_url='./pretrained/VGG.py')
mox.file.copy_parallel(src_url=obs_path+"pretrained/glove.6B.200d.word2vec.txt", dst_url='./pretrained/glove.6B.200d.word2vec.txt')
mox.file.copy_parallel(src_url=obs_path+"pretrained/embeddings.py", dst_url='./pretrained/embeddings.py')
mox.file.copy_parallel(src_url=obs_path+"pretrained/__init__.py", dst_url='./pretrained/__init__.py')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map1.npy", dst_url='./pretrained/feature_map1.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map2.npy", dst_url='./pretrained/feature_map2.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map3.npy", dst_url='./pretrained/feature_map3.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map4.npy", dst_url='./pretrained/feature_map4.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map5.npy", dst_url='./pretrained/feature_map5.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map6.npy", dst_url='./pretrained/feature_map6.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map7.npy", dst_url='./pretrained/feature_map7.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map8.npy", dst_url='./pretrained/feature_map8.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map9.npy", dst_url='./pretrained/feature_map9.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map10.npy", dst_url='./pretrained/feature_map10.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map11.npy", dst_url='./pretrained/feature_map11.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map12.npy", dst_url='./pretrained/feature_map12.npy')
mox.file.copy_parallel(src_url=obs_path+"pretrained/feature_map13.npy", dst_url='./pretrained/feature_map13.npy')
# mox.file.copy_parallel(src_url=obs_path+"data",       dst_url='./data')

INFO:root:Using MoXing-v2.0.0.rc2.4b57a67b-4b57a67b
INFO:root:Using OBS-Python-SDK-3.20.9.1


OSError: [Errno 28] No space left on device

In [3]:
import mindspore
import numpy as np
import os
from easydict import EasyDict
from preprocess.preprocess import *
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # 允许重复载入lib文件

INFO:matplotlib.font_manager:generated new fontManager


图模式

In [4]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

PyNative模式

In [4]:
from mindspore import context
context.set_context(mode=context.PYNATIVE_MODE)

Ascend 环境安装 MindSpore Hub

In [ ]:
! pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/1.3.0/Hub/any/mindspore_hub-1.3.0-py3-none-any.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import mindspore_hub as mshub
from mindspore import context

context.set_context(mode=context.GRAPH_MODE,
                    device_target="Ascend",
                    device_id=0)

model = "mindspore/ascend/1.3/vgg16_cifar10"
network = mshub.load(model)
network.set_train(False)

## 1 预处理

### 1.1 预处理配置

In [5]:
padding = '<pad>'
config = EasyDict({
	'train_img_path': './data/images/train/COCO_train2014_',
	'train_ans_path': './data/annotations/train.json',
	'train_que_path': './data/questions/train.json',
	'valid_img_path': './data/images/val/COCO_val2014_',
	'valid_ans_path': './data/annotations/val.json',
	'valid_que_path': './data/questions/val.json',
	'test_img_path': './data/images/test/COCO_val2014_',
	'test_ans_path':  './data/annotations/test.json',
	'test_que_path':  './data/questions/test.json',
	'max_length': 25,
	'dict_path': './mindrecord/dict.npy',
	'idx_word_dict_path': './mindrecord/idx_word_dict.npy',
	'filter_set_path': './mindrecord/',
	'pretrained_path': './pretrained/',
	'num_splits': 1,
	'train_mindrecord_path': './mindrecord/train.mindrecord',
	'valid_mindrecord_path': './mindrecord/valid.mindrecord',
	'test_mindrecord_path':  './mindrecord/test.mindrecord',
})

### 1.2 读取数据

注: 只取那些答案长度为1的vqa组合

In [27]:
# get 3 types of input data
train_images, train_questions, train_answers, train_options = get_list(config.train_que_path, config.train_ans_path)
valid_images, valid_questions, valid_answers, valid_options = get_list(config.valid_que_path, config.valid_ans_path)
test_images,  test_questions,  test_answers, test_options  = get_list(config.test_que_path,  config.test_ans_path)

In [28]:
total_questions = train_questions + valid_questions + test_questions
total_answers   = train_answers + valid_answers + test_answers
total_options   = train_options + valid_options + test_options

In [29]:
import numpy as np
np.array(train_options).shape

(44375, 10)

### 1.3 构建词典

In [30]:
# build word vocab
word_dict = dict({'<pad>': 0})
word_dict = add_word_into_dict(total_questions, word_dict)
word_dict = add_word_into_dict(total_options, word_dict)

answer_dict = dict()
answer_dict = add_answer_into_dict(total_answers, answer_dict)

In [31]:
# build revert dict
idx_word_dict = dict()
for item in word_dict.items():
	idx_word_dict[item[1]] = item[0]

In [32]:
len(answer_dict)

8193

In [33]:
# save dict
np.save(config.dict_path, word_dict)
np.save(config.idx_word_dict_path, idx_word_dict)

In [ ]:
mox.file.copy_parallel(src_url="./mindrecord/dict.npy", dst_url=obs_path+"mindrecord/dict.npy")
mox.file.copy_parallel(src_url="./mindrecord/idx_word_dict.npy", dst_url=obs_path+"mindrecord/idx_word_dict.npy") 

### 1.4 向量化 & 补齐长度

In [34]:
# word -> vector & padding
train_questions_vec = get_vec_and_pad(train_questions, word_dict, config.max_length)
valid_questions_vec = get_vec_and_pad(valid_questions, word_dict, config.max_length)
test_questions_vec = get_vec_and_pad(test_questions, word_dict, config.max_length)

train_options_vec = get_option_vec_and_pad(train_options, word_dict, 1)
valid_options_vec = get_option_vec_and_pad(valid_options, word_dict, 1)
test_options_vec = get_option_vec_and_pad(test_options, word_dict, 1)

train_answers_vec, _, _, _ = get_answer_to_idx(train_answers, answer_dict)
valid_answers_vec, _, _, _ = get_answer_to_idx(valid_answers, answer_dict)
test_answers_vec, _, _, _ = get_answer_to_idx(test_answers, answer_dict)
_, bool_set, num_set, other_set = get_answer_to_idx(total_answers, answer_dict)

In [13]:
np.array(train_options_vec)

array([[    3,     3,   241, ...,     3,     3,     3],
       [ 2280,  2280,  2280, ...,  2280,  2280,  2280],
       [ 4582,   405,  8566, ...,   126,  1027,  1027],
       ...,
       [ 1415,  1415,  2476, ...,  1415,  1415,  1415],
       [  505,   505,   505, ...,   505, 33781,   505],
       [ 2476,  2476,  2476, ...,  2476,  1415,  2476]])

In [13]:
# 保存集合
np.save(config.filter_set_path+'bool.npy', bool_set)
np.save(config.filter_set_path+'num.npy', num_set)
np.save(config.filter_set_path+'other.npy', other_set)

In [15]:
train_answers

In [14]:
len(other_set)

7864

In [14]:
cnt = 0
for answer in total_answers:
    # print(answer)
    if answer_dict[answer] in other_set:
        cnt += 1
cnt

43740

In [15]:
len(total_answers)

87245

In [16]:
# train_answers_vec

### 1.5 取频率较高的那些词得到答案词集

In [35]:
total_answers_vec = train_answers_vec + valid_answers_vec + test_answers_vec
least_2_set = get_filtered_answer_set(total_answers_vec, 2) # 2958
least_6_set = get_filtered_answer_set(total_answers_vec, 6) # 999

0.9399965614075305
0.8768410797180354


In [18]:
len(least_2_set)

2958

In [19]:
len(least_6_set)

999

In [20]:
# 保存集合
np.save(config.filter_set_path+'min2.npy', least_2_set)
np.save(config.filter_set_path+'min6.npy', least_6_set)

### 1.6 生成MindRecord

train

In [21]:
generate_mindrecord(config.train_mindrecord_path, config.train_img_path, config.num_splits, train_images, train_questions_vec, train_answers_vec, train_options_vec)

train


In [22]:
mox.file.copy_parallel(src_url="./mindrecord/train.mindrecord",    dst_url=obs_path+"mindrecord/train.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/train.mindrecord.db", dst_url=obs_path+"mindrecord/train.mindrecord.db") 

valid

In [18]:
generate_mindrecord(config.valid_mindrecord_path, config.valid_img_path, config.num_splits, valid_images, valid_questions_vec, valid_answers_vec, valid_options_vec)

valid


In [23]:
! ls mindrecord

bool.npy	   min2.npy  num.npy	       train.mindrecord.db
dict.npy	   min5.npy  other.npy	       valid.mindrecord
idx_word_dict.npy  min6.npy  train.mindrecord  valid.mindrecord.db


In [20]:
mox.file.copy_parallel(src_url="./mindrecord/valid.mindrecord",    dst_url=obs_path+"mindrecord/valid.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/valid.mindrecord.db", dst_url=obs_path+"mindrecord/valid.mindrecord.db") 

test

In [21]:
generate_mindrecord(config.test_mindrecord_path, config.test_img_path, config.num_splits, test_images, test_questions_vec, test_answers_vec, test_options_vec)

test


In [22]:
mox.file.copy_parallel(src_url="./mindrecord/test.mindrecord",    dst_url=obs_path+"mindrecord/test.mindrecord") 
mox.file.copy_parallel(src_url="./mindrecord/test.mindrecord.db", dst_url=obs_path+"mindrecord/test.mindrecord.db") 

## 2 加载数据

### 2.1 加载词典、集合

In [6]:
# load dict
word_dict = np.load(config.dict_path, allow_pickle=True).item()
idx_word_dict = np.load(config.idx_word_dict_path, allow_pickle=True).item()

In [7]:
# load filter set
least_2_set = np.load(config.filter_set_path+'min2.npy', allow_pickle=True).item()
least_6_set = np.load(config.filter_set_path+'min6.npy', allow_pickle=True).item()
bool_set = np.load(config.filter_set_path+'bool.npy', allow_pickle=True).item()
num_set = np.load(config.filter_set_path+'num.npy', allow_pickle=True).item()
other_set = np.load(config.filter_set_path+'other.npy', allow_pickle=True).item()
feature_map1_set = np.load(config.pretrained_path+'feature_map1.npy', allow_pickle=True).item()
feature_map2_set = np.load(config.pretrained_path+'feature_map2.npy', allow_pickle=True).item()
feature_map3_set = np.load(config.pretrained_path+'feature_map3.npy', allow_pickle=True).item()
feature_map4_set = np.load(config.pretrained_path+'feature_map4.npy', allow_pickle=True).item()
feature_map5_set = np.load(config.pretrained_path+'feature_map5.npy', allow_pickle=True).item()
feature_map6_set = np.load(config.pretrained_path+'feature_map6.npy', allow_pickle=True).item()
feature_map7_set = np.load(config.pretrained_path+'feature_map7.npy', allow_pickle=True).item()
feature_map8_set = np.load(config.pretrained_path+'feature_map8.npy', allow_pickle=True).item()
feature_map9_set = np.load(config.pretrained_path+'feature_map9.npy', allow_pickle=True).item()
feature_map10_set = np.load(config.pretrained_path+'feature_map10.npy', allow_pickle=True).item()
feature_map11_set = np.load(config.pretrained_path+'feature_map11.npy', allow_pickle=True).item()
feature_map12_set = np.load(config.pretrained_path+'feature_map12.npy', allow_pickle=True).item()
feature_map13_set = np.load(config.pretrained_path+'feature_map13.npy', allow_pickle=True).item()

FileNotFoundError: [Errno 2] No such file or directory: './pretrained/feature_map1.npy'

In [17]:
a = feature_map1_set + feature_map2_set

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [73]:
# set -> dict
least_2_dict = set_to_dict(least_2_set)
least_6_dict = set_to_dict(least_6_set)

bool_dict = set_to_dict(bool_set)
num_dict = set_to_dict(num_set)
other_dict = set_to_dict(other_set)

In [74]:
feature_map1_set

# for key, value in feature_map1_set.items():
#     print(key)
#     print(value)
#     print(value.shape)
#     break

least_2_dict


{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 19: 17,
 20: 18,
 22: 19,
 23: 20,
 25: 21,
 26: 22,
 27: 23,
 28: 24,
 29: 25,
 30: 26,
 31: 27,
 32: 28,
 33: 29,
 34: 30,
 35: 31,
 36: 32,
 37: 33,
 38: 34,
 39: 35,
 40: 36,
 41: 37,
 42: 38,
 43: 39,
 44: 40,
 45: 41,
 46: 42,
 47: 43,
 48: 44,
 49: 45,
 50: 46,
 51: 47,
 52: 48,
 54: 49,
 55: 50,
 56: 51,
 57: 52,
 58: 53,
 59: 54,
 60: 55,
 61: 56,
 63: 57,
 64: 58,
 65: 59,
 66: 60,
 67: 61,
 70: 62,
 72: 63,
 73: 64,
 74: 65,
 75: 66,
 76: 67,
 77: 68,
 78: 69,
 79: 70,
 80: 71,
 81: 72,
 82: 73,
 83: 74,
 84: 75,
 85: 76,
 86: 77,
 89: 78,
 90: 79,
 91: 80,
 92: 81,
 94: 82,
 95: 83,
 96: 84,
 97: 85,
 98: 86,
 100: 87,
 102: 88,
 103: 89,
 105: 90,
 106: 91,
 108: 92,
 109: 93,
 110: 94,
 111: 95,
 112: 96,
 114: 97,
 115: 98,
 116: 99,
 117: 100,
 118: 101,
 119: 102,
 120: 103,
 121: 104,
 122: 105,
 123: 106,
 124: 107,
 126: 108,
 127: 1

### 2.2 训练配置

In [43]:
# model_name = 'baseline'
# model_name = 'stack_attention'
model_name = 'topdown_attention'

In [44]:
# batch_size = 32 # baseline
# batch_size = 100 # stacked attention
batch_size = 128 # top-down attention

In [12]:
! pip install gensim

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [48]:
train_config = EasyDict({
	'model': model_name,
	'vocab_size': 52083,
	'output_size': 999 if model_name == 'baseline' or model_name == 'stack_attention' else 2958,
	'batch_size': batch_size,
	'epoch_size': 3,
	'max_length': 14,
	'use_op': False,
	'use_pretrained_feature_map': True,
	'hidden_size': 1024,
	'lr': 1e-4,
	'momentum': 0.9,
	'weight_decay': 3e-5,
	'early_stop': 100,
	'ckpt_save_path': './ckpt',
	'checkpoint_path': './ckpt/'+model_name+'.ckpt',
	'pretrained_path': './pretrained/vgg19_ascend_v130_imagenet2012_research_cv_top1acc74_top5acc91.97.ckpt',
	'embedding_table_path': './pretrained/embedding_table_glove_200d.txt',
	'glove_vector_path': './pretrained/glove.6B.200d.txt',
	'glove_word2vec_path': './pretrained/glove.6B.200d.word2vec.txt',
})

### 2.3 生成数据集

In [49]:
# platform = 'Local'
# platform = 'Colob'
platform = 'Ascend'

In [60]:
# frequency filter dict
filter_dict = least_6_dict if model_name == 'baseline' or model_name == 'stack_attention' else least_2_dict
# image size
image_width = 224 if model_name == 'baseline' else 448
image_height = 224 if model_name == 'baseline' else 448
# parallel workers
num_parallel_workers = 4 if platform == 'Ascend' or platform == 'Local' else 2
print(num_parallel_workers)
# create dataset
train_dataset = generate_dataset(config.train_mindrecord_path, train_config.batch_size, 1, train_config.max_length, 
                                 filter_dict, image_height, image_width, num_parallel_workers)
valid_dataset = generate_dataset(config.valid_mindrecord_path, train_config.batch_size, 1, train_config.max_length, 
                                 filter_dict, image_height, image_width, num_parallel_workers)
test_dataset  = generate_dataset(config.test_mindrecord_path , train_config.batch_size, 1, train_config.max_length, 
                                 filter_dict, image_height, image_width, num_parallel_workers)

4


## 3 训练模型

### 3.1 创建模型

In [62]:
import mindspore.nn as nn
import mindspore.ops.operations as P
from utils.metric_utils import *
from utils.wrapper_utils import *
from utils.callback_utils import *

In [63]:
class Network(nn.Cell):
	def __init__(self, train_config):
		super(Network, self).__init__()
		self.reshape = P.Reshape()
		self.embedding = nn.Embedding(train_config.vocab_size, train_config.hidden_size)
		self.out = nn.Dense(train_config.hidden_size*train_config.max_length, train_config.output_size)
	def construct(self, images, questions):
		x = self.embedding(questions)
		x = x.reshape(x.shape[0], -1)
		x = self.out(x)
		return x

In [64]:
from model.vqa_baseline import *
from model.stack_attention import *
from model.topdown_attention import *

In [65]:
# 创建网络
if model_name == 'baseline':
	if train_config.use_op:
		network = VQABasicOpAttn(train_config)
	else:
		network = VQABasic(train_config)
elif model_name == 'stack_attention':
	if train_config.use_op:
		network = StackedAttentionNetOpAttn(word_dict, train_config)
	else:
		network = StackedAttentionNet(word_dict, train_config)
elif model_name == 'topdown_attention':
	if train_config.use_op and train_config.use_pretrained_feature_map:
		network = TopDownAttentionNetOpAttn(word_dict, train_config)
	elif train_config.use_op:
		network = TopDownAttentionNetOpAttn(word_dict, train_config)
	elif train_config.use_pretrained_feature_map:
		network = TopDownAttentionNetFeature(word_dict, train_config)
	else:
		network = TopDownAttentionNet(word_dict, train_config)

In [46]:
# questions = Tensor(np.zeros((32, 14), dtype=np.int32))
# images = Tensor(np.zeros((32, 448, 448, 3), dtype=np.int8))
# options = Tensor(np.zeros((32, 10), dtype=np.int32))
# output = network(images, questions, options)
# print(output)

In [66]:
# 创建训练、测试网络
loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True)
# train network
train_net = TrainNetworkWrapper(network, loss_fn, train_config)
train_net.set_train(True)
# valid network
valid_net = WithEvalCellWrapper(network, train_config)
valid_net.set_train(False)

WithEvalCellWrapper<
  (network): TopDownAttentionNetFeature<
    (text_net): TextFeatureCell<
      (embeddings): Embedding<vocab_size=52083, embedding_size=200, use_one_hot=False, embedding_table=Parameter (name=text_net.embeddings.embedding_table, shape=(52083, 200), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
      >
    (fc_feature): Dense<
      input_channels=8192, output_channels=2048, has_bias=True, activation=Tanh<>
      (activation): Tanh<>
      >
    (topdown_attn): TopDownAttentionLayer<
      (fc_attn_nonlin): Dense<
        input_channels=2560, output_channels=512, activation=Tanh<>
        (activation): Tanh<>
        >
      (fc_attn_lin): Dense<input_channels=512, output_channels=1>
      >
    (fc_questions): Dense<
      input_channels=512, output_channels=512, has_bias=True, activation=Tanh<>
      (activation): Tanh<>
      >
    (fc_images): Dense<
      input_channels=2048, output_channels=512, has_bias=True, activation=Tanh<>
      (a

### 3.2 开始训练

In [67]:
def train(train_net, valid_net, train_dataset, valid_dataset, train_config):
	# 创建文件夹
	if not os.path.exists(train_config.ckpt_save_path):
		os.mkdir(train_config.ckpt_save_path)
	
	current_step = 0
	valid_acc_max = 0.0
	valid_loss_min = np.inf
	valid_acc_model = 0
	valid_loss_model = np.inf
	for epoch_num in range(1, train_config.epoch_size+1):
		# train
		train_losses = []
		train_accs = []
		for i in train_dataset.create_dict_iterator():
			if train_config.use_pretrained_feature_map:
				train_loss, train_acc = train_net(i['image_id'], i['question'], i['answer'], i['options'])
			else:
				train_loss, train_acc = train_net(i['image'], i['question'], i['answer'], i['options'])
			train_losses.append(train_loss.item(0).asnumpy().item())
			train_accs.append(train_acc.item(0).asnumpy().item())
		train_loss = sum(train_losses) / len(train_losses)
		train_acc = sum(train_accs) / len(train_accs)
		print('epoch:', epoch_num, ' train loss =', train_loss, 'acc =', train_acc)

		# valid
		loss = []
		acc = []
		for j in valid_dataset.create_dict_iterator():
			if train_config.use_pretrained_feature_map:
				step_loss, step_acc = valid_net(j['image_id'], j['question'], j['answer'], j['options'])
			else:
				step_loss, step_acc = valid_net(j['image'], j['question'], j['answer'], j['options'])
			loss.append(step_loss.item(0).asnumpy().item())
			acc.append(step_acc.item(0).asnumpy().item())
		valid_loss = sum(loss) / len(loss)
		valid_acc = sum(acc) / len(acc)
		print('-- valid loss =', valid_loss, 'acc =', valid_acc)
		
		# save ckpt / early stop
		if valid_acc >= valid_acc_max or valid_loss < valid_loss_min:
			if valid_acc >= valid_acc_max and valid_loss < valid_loss_min:
				valid_acc_model = valid_acc
				valid_loss_model = valid_loss
				save_checkpoint(valid_net.network, train_config.checkpoint_path)
				valid_acc_max = np.max((valid_acc_max, valid_acc))
				valid_loss_min = np.min((valid_loss_min, valid_loss))
			current_step = 0
		else:
			current_step += 1
			if current_step == train_config.early_stop:
				print('early stop... min loss:', valid_loss_min, 'max acc:', valid_acc_max, end='')
				print('; validation model loss:', valid_loss_model, 'acc:', valid_acc_model)

In [68]:
train(train_net, valid_net, train_dataset, valid_dataset, train_config)

RuntimeError: Unexpected error. Invalid data, column name: image_id can not found in schema. Please check the 'column_list'.
Line of code : 185
File         : D:\jenkins\agent-working-dir\workspace\Compile_CPU_Windows_PY38\mindspore\mindspore\ccsrc\minddata\mindrecord\io\shard_reader.cc


In [ ]:
mox.file.copy_parallel(src_url=train_config.checkpoint_path, dst_url=obs_path+train_config.checkpoint_path) 

## 4 测试模型

### 4.1 创建测试模型

In [ ]:
from mindspore import load_checkpoint
from model.vqa_baseline import *
from model.stack_attention import *
from model.topdown_attention import *

In [ ]:
# 创建网络
if model_name == 'baseline':
	if train_config.use_op:
		network = VQABasicOpAttn(train_config)
	else:
		network = VQABasic(train_config)
elif model_name == 'stack_attention':
	if train_config.use_op:
		network = StackedAttentionNetOpAttn(word_dict, train_config)
	else:
		network = StackedAttentionNet(word_dict, train_config)
elif model_name == 'topdown_attention':
	if train_config.use_op:
		network = TopDownAttentionNetOpAttn(word_dict, train_config)
	else:
		network = TopDownAttentionNet(word_dict, train_config)

In [ ]:
load_checkpoint(train_config.checkpoint_path, net=network)
test_net = WithEvalCellWrapper(network, train_config)
test_net.set_train(False)

WithEvalCellWrapper<
  (network): TopDownAttentionNet<
    (text_net): TextFeatureCell<
      (embeddings): Embedding<vocab_size=10233, embedding_size=200, use_one_hot=False, embedding_table=Parameter (name=text_net.embeddings.embedding_table, shape=(10233, 200), dtype=Float32, requires_grad=True), dtype=Float32, padding_idx=None>
      >
    (VGGnet): VGG<
      (conv1): Conv2d<input_channels=3, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=Parameter (name=VGGnet.conv1.weight_init, shape=(64, 3, 3, 3), dtype=Float32, requires_grad=False), bias_init=zeros, format=NCHW>
      (conv2): Conv2d<input_channels=64, output_channels=64, kernel_size=(3, 3), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=Parameter (name=VGGnet.conv2.weight_init, shape=(64, 64, 3, 3), dtype=Float32, requires_grad=False), bias_init=zeros, format=NCHW>
      (conv3): Conv2d<input_

### 4.2 开始测试

In [ ]:
def test(test_net, test_dataset):
	# test
	loss = []
	acc = []
	for i in test_dataset.create_dict_iterator():
		if train_config.use_pretrained_feature_map:
			step_loss, step_acc = test_net(i['image_id'], i['question'], i['answer'], i['options'])
		else:
			step_loss, step_acc = test_net(i['image'], i['question'], i['answer'], i['options'])
		loss.append(step_loss.item(0).asnumpy().item())
		acc.append(step_acc.item(0).asnumpy().item())
	test_loss = sum(loss) / len(loss)
	test_acc = sum(acc) / len(acc)
	print('test loss =', test_loss, 'acc =', test_acc)

In [ ]:
test(test_net, test_dataset)

test loss = 6.822631631798814 acc = 0.20789579967689822


以下为随便测试区域

In [ ]:
pretrained_path = './pretrained/fasterrcnnresnetv1152_ascend_v130_coco2017_official_cv_mAP41.1.ckpt'
params = load_checkpoint(pretrained_path)

In [ ]:
params

NameError: name 'params' is not defined

In [ ]:
import pandas as pd
read_file = pd.read_csv("f:/Desk/test2014_36/test2014_resnet101_faster_rcnn_genome_36.tsv", sep = '\t')

ParserError: Error tokenizing data. C error: out of memory

In [ ]:
read_file.loc[0]